In [1]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten,Subtract, Lambda, Concatenate
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, Accuracy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
SIZE = 200

In [4]:

def get_class_file_paths(path):
    classes = os.listdir(path)
    file_paths = {}
    for class_name in classes:
        class_path = os.path.join(path, class_name)
        file_paths[class_name] = tf.data.Dataset.list_files(class_path + '/*.jpg')
    return file_paths

In [5]:
def preprocess(file_path):

    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)

    img = tf.image.resize(img, (SIZE,SIZE))
    img = img / 255.0

    return img

In [6]:
def preprocess_twin(input_img, validation_img, label):
    return (preprocess(input_img), preprocess(validation_img), label)


In [7]:
def make_embedding():
    inp = Input(shape=(SIZE,SIZE,1), name='input_image')

    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)

    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)

    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)

    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)


    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [8]:
embedding = make_embedding()

In [9]:
# Siamese L1 Distance class
class L1Dist(Layer):

    def __init__(self, **kwargs):
        super().__init__()

    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [10]:
# Reload model
siamese_model = tf.keras.models.load_model('/content/drive/MyDrive/log_id_project/model/siamesemodel_gray_cnn.h5',
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [50]:
#input image path, reference image where we need to verify the identity. The image should me preprocessed and grayscale
input_path = '/content/drive/MyDrive/log_id_project/data/database/input/input_pine4.jpg'
#folder path where the tree images locate. One image per each tree. tree name is the file name
validation_path = '/content/drive/MyDrive/log_id_project/data/database/verification'

In [51]:
def verify(model,detection_threshold):
    results = {}
    for image in os.listdir(os.path.join(validation_path)):
        input_img = preprocess(os.path.join(input_path))
        validation_img = preprocess(os.path.join(validation_path, image))
        tree_name = os.path.splitext(os.path.basename(image))[0]
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        if result > detection_threshold:
            results[tree_name] = result

    print(results)


    if len(results)==1:
      end_result = f"This tree is {list(results.keys())[0]}"
    elif len(results)>1:
      end_result = f"This tree is {max(results, key=results.get)}"
    else:
      end_result = "This tree is not in the database"


    return end_result

In [52]:
end_result = verify(siamese_model,0.5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
{'pine 4': array([[0.84521914]], dtype=float32)}


In [53]:
end_result

'This tree is pine 4'